#Beatles-like Poetry Generation

In this notebook we will use Beatles songs and LSTM to generate poems that sound a bit like Beatles

You may use other/more data to train the model and get other results! :)



In [25]:
import tensorflow as tf
import string
import requests
import pandas as pd


In [28]:
response = requests.get('https://raw.githubusercontent.com/h4nkyn/beatlestxt/master/beatles.txt')
data = response.text.splitlines()
print("Length of data: ",len(data))

Length of data:  6613


#Build the LSTM Model and Prepare X and y

In [29]:
import numpy as numpy
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [30]:
#doing tokenization

token = Tokenizer()
token.fit_on_texts(data)

In [31]:
encoded_text = token.texts_to_sequences(data)

vocab_size = len(token.word_counts) + 1

#Let's prepare training data

In [32]:
datalist = []
for d in encoded_text:
  if len(d)>1:
    for i in range(2, len(d)):
      datalist.append(d[:i])

#Padding

In [33]:
max_length = 20
sequences = pad_sequences(datalist, maxlen=max_length, padding='pre')
X = sequences[:, :-1]
y = sequences[:, -1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]

#LSTM Model Training

In [34]:
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

model.fit(X, y, batch_size=32, epochs=50)

Epoch 1/50
744/744 [==============================] - 17s 19ms/step - loss: 5.9757 - accuracy: 0.0433
Epoch 2/50
744/744 [==============================] - 14s 19ms/step - loss: 5.5565 - accuracy: 0.0526
Epoch 3/50
744/744 [==============================] - 14s 18ms/step - loss: 5.3185 - accuracy: 0.0665
Epoch 4/50
744/744 [==============================] - 14s 19ms/step - loss: 5.1576 - accuracy: 0.0808
Epoch 5/50
744/744 [==============================] - 14s 19ms/step - loss: 5.0256 - accuracy: 0.0884
Epoch 6/50
744/744 [==============================] - 14s 18ms/step - loss: 4.8911 - accuracy: 0.1025
Epoch 7/50
744/744 [==============================] - 14s 18ms/step - loss: 4.7507 - accuracy: 0.1161
Epoch 8/50
744/744 [==============================] - 14s 19ms/step - loss: 4.6127 - accuracy: 0.1275
Epoch 9/50
744/744 [==============================] - 14s 18ms/step - loss: 4.4868 - accuracy: 0.1383
Epoch 10/50
744/744 [==============================] - 14s 18ms/step - loss: 4.370

In [35]:
poetry_length = 10
import numpy as np

def generate_poetry(seed_text, n_lines):
  for i in range(n_lines):
    text = []
    for _ in range(poetry_length):
      encoded = token.texts_to_sequences([seed_text])
      encoded = pad_sequences(encoded, maxlen=seq_length, padding = 'pre')
      y_pred = np.argmax(model.predict(encoded), axis=-1)

      predicted_word = ""
      for word, index in token.word_index.items():
        if index == y_pred:
          predicted_word = word
          break

      seed_text = seed_text + " " + predicted_word
      text.append(predicted_word)
    
    seed_text = text[-1]
    text = " ".join(text)
    print(text)

In [40]:
seed_text = "You are cute"
generate_poetry(seed_text, 5)

tearing me like you did the night and i begged
a chance my trust and i can make it easy
seems to see you buy you to be mine i
said i’m trav’ling on the one after 9 guaranteed on
the beginning of the dark black his mother nature’s waiting
